# Bayesian Inference on Impedance Spectra
## Circuit Model Selection Using Bayesian Statistics

**Objective:** Use Bayesian inference to determine which circuit model (Circuit A or Circuit B) best fits the measured impedance spectroscopy data.

---

## Table of Contents
1. [Introduction and Theoretical Background](#introduction)
2. [Data Loading and Exploration](#data-loading)
3. [Circuit Models and Impedance Formulations](#circuit-models)
4. [Bayesian Inference Framework](#bayesian-framework)
5. [Parameter Estimation](#parameter-estimation)
6. [Model Comparison](#model-comparison)
7. [Sensor Uncertainties and Error Propagation](#uncertainties)
8. [Data Fusion and Architecture](#data-fusion)
9. [Critical Analysis and Sensitivity](#sensitivity)
10. [Conclusions](#conclusions)

## 1. Introduction and Theoretical Background <a name="introduction"></a>

### AC Impedance Spectroscopy
AC impedance spectroscopy measures the complex impedance of a circuit as a function of frequency. The impedance $Z(\omega)$ is a complex quantity:

$$Z(\omega) = Z_{\text{real}}(\omega) + jZ_{\text{imag}}(\omega)$$

where $\omega = 2\pi f$ is the angular frequency.

### Circuit Models

**Circuit A:** Simple RC parallel circuit
- Contains a resistor R in parallel with a capacitor C
- Impedance: $Z_A(\omega) = \frac{R}{1 + j\omega RC}$

**Circuit B:** Two parallel RC branches
- Contains two parallel branches, each with R and C in parallel
- Impedance: $Z_B(\omega) = \frac{1}{\frac{1}{Z_1} + \frac{1}{Z_2}}$ where $Z_i = \frac{R_i}{1 + j\omega R_i C_i}$

### Bayesian Inference Approach

We apply Bayes' theorem for model selection:

$$P(M_i | D) = \frac{P(D | M_i) P(M_i)}{P(D)}$$

where:
- $P(M_i | D)$ is the posterior probability of model $i$ given data $D$
- $P(D | M_i)$ is the likelihood of data given model $i$ (evidence)
- $P(M_i)$ is the prior probability of model $i$
- $P(D)$ is the marginal likelihood (normalizing constant)

For parameter estimation within each model:

$$P(\theta | D, M_i) = \frac{P(D | \theta, M_i) P(\theta | M_i)}{P(D | M_i)}$$

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.optimize import minimize, differential_evolution
from scipy.integrate import simpson
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

print("Libraries imported successfully!")

## 2. Data Loading and Exploration <a name="data-loading"></a>

The data is stored in .npz files containing impedance measurements at different frequencies.
- Frequency range: 0 to 12501 Hz
- Step size: 250 Hz
- First item in each list is the measured impedance

In [ ]:
# Load data from Circuit1 and Circuit2
circuit1_data = np.load('Circuit1/spectrum-f_min_0-f_max_12501-f_step_250.npz', allow_pickle=True)
circuit2_data = np.load('Circuit2/spectrum-f_min_0-f_max_12501-f_step_250.npz', allow_pickle=True)

# Extract data
print("Circuit1 data keys:", circuit1_data.files)
print("Circuit2 data keys:", circuit2_data.files)

# Extract impedance measurements
# Based on the assignment, the first item is the measured impedance
for key in circuit1_data.files:
    data1 = circuit1_data[key]
    print(f"\nCircuit1 - {key}:")
    print(f"  Type: {type(data1)}")
    print(f"  Shape/Length: {len(data1) if hasattr(data1, '__len__') else 'N/A'}")
    if hasattr(data1, '__len__') and len(data1) > 0:
        print(f"  First element type: {type(data1[0])}")
        print(f"  First few elements: {data1[:3]}")

In [ ]:
# Extract frequencies and impedances
# The data structure needs to be examined
def extract_impedance_data(data_file):
    """
    Extract frequency and impedance from the loaded .npz file
    """
    # Get the array from the file
    key = data_file.files[0]
    data_array = data_file[key]
    
    # The first element should be impedance
    # Data is collected from 0 to 12501 Hz with step 250
    frequencies = np.arange(0, 12501, 250)
    
    # Extract impedance (should be complex numbers)
    impedances = data_array
    
    return frequencies, impedances

freq1, Z1 = extract_impedance_data(circuit1_data)
freq2, Z2 = extract_impedance_data(circuit2_data)

print(f"Frequency array shape: {freq1.shape}")
print(f"Frequency range: {freq1[0]} Hz to {freq1[-1]} Hz")
print(f"Number of frequency points: {len(freq1)}")
print(f"\nCircuit1 impedance shape: {Z1.shape}")
print(f"Circuit2 impedance shape: {Z2.shape}")
print(f"\nFirst few Circuit1 impedance values: {Z1[:5]}")

In [ ]:
# Visualize the impedance data - Nyquist plot
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Convert to complex if needed
if Z1.dtype == 'object':
    # Handle object arrays - extract the first element if it's nested
    if hasattr(Z1[0], '__len__') and not isinstance(Z1[0], complex):
        Z1_complex = np.array([complex(z[0]) if hasattr(z, '__len__') else complex(z) for z in Z1])
        Z2_complex = np.array([complex(z[0]) if hasattr(z, '__len__') else complex(z) for z in Z2])
    else:
        Z1_complex = np.array([complex(z) if isinstance(z, (int, float, complex)) else z[0] for z in Z1])
        Z2_complex = np.array([complex(z) if isinstance(z, (int, float, complex)) else z[0] for z in Z2])
elif Z1.ndim > 1:
    # If it's a 2D array, take the first column
    Z1_complex = Z1[:, 0]
    Z2_complex = Z2[:, 0]
else:
    Z1_complex = Z1
    Z2_complex = Z2

# Ensure they are 1D complex arrays
Z1_complex = np.squeeze(Z1_complex)
Z2_complex = np.squeeze(Z2_complex)

print(f"After processing:")
print(f"Z1_complex shape: {Z1_complex.shape}")
print(f"Z2_complex shape: {Z2_complex.shape}")
print(f"Z1_complex dtype: {Z1_complex.dtype}")
print(f"First few Z1_complex values: {Z1_complex[:3]}")

# Nyquist plots
axes[0, 0].plot(Z1_complex.real, -Z1_complex.imag, 'o-', markersize=4, label='Circuit 1')
axes[0, 0].set_xlabel('Real(Z) [Ω]')
axes[0, 0].set_ylabel('-Imag(Z) [Ω]')
axes[0, 0].set_title('Circuit 1 - Nyquist Plot')
axes[0, 0].grid(True)
axes[0, 0].axis('equal')

axes[0, 1].plot(Z2_complex.real, -Z2_complex.imag, 'o-', markersize=4, color='orange', label='Circuit 2')
axes[0, 1].set_xlabel('Real(Z) [Ω]')
axes[0, 1].set_ylabel('-Imag(Z) [Ω]')
axes[0, 1].set_title('Circuit 2 - Nyquist Plot')
axes[0, 1].grid(True)
axes[0, 1].axis('equal')

# Bode plots - Magnitude
axes[1, 0].semilogx(freq1[1:], np.abs(Z1_complex[1:]), 'o-', markersize=4, label='Circuit 1')
axes[1, 0].semilogx(freq2[1:], np.abs(Z2_complex[1:]), 'o-', markersize=4, label='Circuit 2')
axes[1, 0].set_xlabel('Frequency [Hz]')
axes[1, 0].set_ylabel('|Z| [Ω]')
axes[1, 0].set_title('Bode Plot - Magnitude')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Bode plots - Phase
axes[1, 1].semilogx(freq1[1:], np.angle(Z1_complex[1:], deg=True), 'o-', markersize=4, label='Circuit 1')
axes[1, 1].semilogx(freq2[1:], np.angle(Z2_complex[1:], deg=True), 'o-', markersize=4, label='Circuit 2')
axes[1, 1].set_xlabel('Frequency [Hz]')
axes[1, 1].set_ylabel('Phase [degrees]')
axes[1, 1].set_title('Bode Plot - Phase')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

print(f"\nData Summary:")
print(f"Circuit 1 - |Z| range: {np.abs(Z1_complex).min():.2f} to {np.abs(Z1_complex).max():.2f} Ω")
print(f"Circuit 2 - |Z| range: {np.abs(Z2_complex).min():.2f} to {np.abs(Z2_complex).max():.2f} Ω")

## 3. Circuit Models and Impedance Formulations <a name="circuit-models"></a>

### Circuit A: Single RC Parallel

The impedance of a parallel RC circuit is:

$$Z_A(\omega) = \frac{R}{1 + j\omega RC} = \frac{R}{1 + j\omega R C}$$

This can be written in terms of real and imaginary parts:

$$Z_A(\omega) = \frac{R}{1 + (\omega RC)^2} - j\frac{\omega R^2 C}{1 + (\omega RC)^2}$$

**Parameters:** $\theta_A = [R, C]$

### Circuit B: Two Parallel RC Branches

For two parallel branches:

$$Z_B(\omega) = \left(\frac{1}{Z_1} + \frac{1}{Z_2}\right)^{-1}$$

where each branch has impedance:

$$Z_i(\omega) = \frac{R_i}{1 + j\omega R_i C_i}$$

**Parameters:** $\theta_B = [R_1, C_1, R_2, C_2]$

In [ ]:
def impedance_model_A(frequencies, R, C):
    """
    Calculate impedance for Circuit A (single RC parallel)
    
    Parameters:
    -----------
    frequencies : array
        Frequency array in Hz
    R : float
        Resistance in Ohms
    C : float
        Capacitance in Farads
    
    Returns:
    --------
    Z : complex array
        Complex impedance
    """
    omega = 2 * np.pi * frequencies
    # Avoid division by zero at f=0
    omega = np.where(omega == 0, 1e-10, omega)
    
    # Z = R / (1 + j*omega*R*C)
    Z = R / (1 + 1j * omega * R * C)
    return Z


def impedance_model_B(frequencies, R1, C1, R2, C2):
    """
    Calculate impedance for Circuit B (two parallel RC branches)
    
    Parameters:
    -----------
    frequencies : array
        Frequency array in Hz
    R1, C1 : float
        Resistance and capacitance of first branch
    R2, C2 : float
        Resistance and capacitance of second branch
    
    Returns:
    --------
    Z : complex array
        Complex impedance
    """
    omega = 2 * np.pi * frequencies
    omega = np.where(omega == 0, 1e-10, omega)
    
    # Impedance of each branch
    Z1 = R1 / (1 + 1j * omega * R1 * C1)
    Z2 = R2 / (1 + 1j * omega * R2 * C2)
    
    # Total impedance (parallel combination)
    Z = 1 / (1/Z1 + 1/Z2)
    return Z


# Test the models with example parameters
test_freq = np.logspace(0, 5, 100)
test_ZA = impedance_model_A(test_freq, R=1000, C=1e-6)
test_ZB = impedance_model_B(test_freq, R1=1000, C1=1e-6, R2=5000, C2=1e-7)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(test_ZA.real, -test_ZA.imag, 'o-', label='Model A')
axes[0].plot(test_ZB.real, -test_ZB.imag, 's-', label='Model B')
axes[0].set_xlabel('Real(Z) [Ω]')
axes[0].set_ylabel('-Imag(Z) [Ω]')
axes[0].set_title('Example Circuit Models - Nyquist Plot')
axes[0].legend()
axes[0].grid(True)

axes[1].semilogx(test_freq, np.abs(test_ZA), 'o-', label='Model A')
axes[1].semilogx(test_freq, np.abs(test_ZB), 's-', label='Model B')
axes[1].set_xlabel('Frequency [Hz]')
axes[1].set_ylabel('|Z| [Ω]')
axes[1].set_title('Example Circuit Models - Bode Magnitude')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

print("Circuit models defined successfully!")

## 4. Bayesian Inference Framework <a name="bayesian-framework"></a>

### Prior Distributions

Based on the assignment specifications:
- **Resistors:** 100 Ω to 10 kΩ → We use log-uniform priors
- **Capacitors:** 10 nF to 10 μF → We use log-uniform priors

Log-uniform priors are appropriate for parameters that span several orders of magnitude:

$$p(\theta) \propto \frac{1}{\theta} \quad \text{for } \theta \in [\theta_{\min}, \theta_{\max}]$$

### Likelihood Function

Assuming Gaussian measurement noise with standard deviation $\sigma$:

$$P(D | \theta, M) = \prod_{i=1}^{N} \frac{1}{2\pi\sigma^2} \exp\left(-\frac{|Z_i^{\text{meas}} - Z_i^{\text{model}}(\theta)|^2}{2\sigma^2}\right)$$

In log-space (more numerically stable):

$$\log P(D | \theta, M) = -\frac{N}{2}\log(2\pi\sigma^2) - \frac{1}{2\sigma^2}\sum_{i=1}^{N}|Z_i^{\text{meas}} - Z_i^{\text{model}}(\theta)|^2$$

### Posterior Distribution

$$P(\theta | D, M) \propto P(D | \theta, M) \times P(\theta | M)$$

In [ ]:
class BayesianCircuitModel:
    """
    Bayesian inference for circuit impedance models
    """
    
    def __init__(self, frequencies, Z_measured, model_type='A'):
        """
        Initialize the Bayesian model
        
        Parameters:
        -----------
        frequencies : array
            Measurement frequencies
        Z_measured : complex array
            Measured impedance values
        model_type : str
            'A' for single RC, 'B' for dual RC
        """
        self.frequencies = frequencies
        self.Z_measured = Z_measured
        self.model_type = model_type
        
        # Prior bounds (from assignment)
        self.R_min, self.R_max = 100, 10000  # Ohms
        self.C_min, self.C_max = 10e-9, 10e-6  # Farads
        
        # Noise model - estimate from data or set manually
        self.sigma = None  # Will be estimated
        
    def log_prior(self, params):
        """
        Log prior probability (log-uniform for R and C)
        """
        if self.model_type == 'A':
            R, C = params
            if not (self.R_min <= R <= self.R_max and self.C_min <= C <= self.C_max):
                return -np.inf
            # Log-uniform prior
            log_p = -np.log(R) - np.log(C)
            log_p -= np.log(np.log(self.R_max/self.R_min))
            log_p -= np.log(np.log(self.C_max/self.C_min))
            return log_p
        
        elif self.model_type == 'B':
            R1, C1, R2, C2 = params
            if not (self.R_min <= R1 <= self.R_max and self.C_min <= C1 <= self.C_max and
                    self.R_min <= R2 <= self.R_max and self.C_min <= C2 <= self.C_max):
                return -np.inf
            # Log-uniform prior
            log_p = -np.log(R1) - np.log(C1) - np.log(R2) - np.log(C2)
            log_p -= 2 * np.log(np.log(self.R_max/self.R_min))
            log_p -= 2 * np.log(np.log(self.C_max/self.C_min))
            return log_p
    
    def log_likelihood(self, params, sigma=None):
        """
        Log likelihood of the data given parameters
        """
        # Calculate model prediction
        if self.model_type == 'A':
            R, C = params
            Z_model = impedance_model_A(self.frequencies, R, C)
        elif self.model_type == 'B':
            R1, C1, R2, C2 = params
            Z_model = impedance_model_B(self.frequencies, R1, C1, R2, C2)
        
        # Calculate residuals (complex)
        residuals = self.Z_measured - Z_model
        
        # Use magnitude of residuals
        residuals_mag = np.abs(residuals)
        
        # Estimate sigma if not provided
        if sigma is None:
            sigma = np.std(residuals_mag)
        
        # Gaussian likelihood
        N = len(residuals)
        log_L = -N/2 * np.log(2*np.pi*sigma**2) - np.sum(residuals_mag**2) / (2*sigma**2)
        
        return log_L
    
    def log_posterior(self, params, sigma=None):
        """
        Log posterior probability
        """
        log_prior = self.log_prior(params)
        if np.isinf(log_prior):
            return -np.inf
        
        log_likelihood = self.log_likelihood(params, sigma)
        return log_prior + log_likelihood
    
    def negative_log_posterior(self, params, sigma=None):
        """
        Negative log posterior for optimization
        """
        return -self.log_posterior(params, sigma)

print("Bayesian framework defined successfully!")

## 5. Parameter Estimation <a name="parameter-estimation"></a>

We use Maximum A Posteriori (MAP) estimation to find the most likely parameters for each model.
We employ differential evolution for global optimization followed by local refinement.

In [ ]:
def estimate_parameters(frequencies, Z_measured, model_type, n_trials=5):
    """
    Estimate parameters using MAP estimation
    """
    model = BayesianCircuitModel(frequencies, Z_measured, model_type)
    
    # Define bounds for optimization
    if model_type == 'A':
        bounds = [(model.R_min, model.R_max), (model.C_min, model.C_max)]
    else:
        bounds = [(model.R_min, model.R_max), (model.C_min, model.C_max),
                  (model.R_min, model.R_max), (model.C_min, model.C_max)]
    
    # Use differential evolution for global optimization
    print(f"Optimizing Model {model_type} parameters...")
    result = differential_evolution(
        lambda params: model.negative_log_posterior(params),
        bounds=bounds,
        maxiter=1000,
        popsize=15,
        seed=42,
        polish=True,
        workers=1
    )
    
    params_opt = result.x
    log_post = -result.fun
    
    print(f"Optimization completed for Model {model_type}")
    print(f"Log posterior: {log_post:.2f}")
    
    if model_type == 'A':
        print(f"  R = {params_opt[0]:.2f} Ω")
        print(f"  C = {params_opt[1]*1e9:.2f} nF")
    else:
        print(f"  R1 = {params_opt[0]:.2f} Ω")
        print(f"  C1 = {params_opt[1]*1e9:.2f} nF")
        print(f"  R2 = {params_opt[2]:.2f} Ω")
        print(f"  C2 = {params_opt[3]*1e9:.2f} nF")
    
    return params_opt, log_post, model

# Estimate parameters for Circuit1 data
print("=" * 60)
print("CIRCUIT 1 - Parameter Estimation")
print("=" * 60)

params_1A, logpost_1A, model_1A = estimate_parameters(freq1, Z1_complex, 'A')
print()
params_1B, logpost_1B, model_1B = estimate_parameters(freq1, Z1_complex, 'B')

print("\n" + "=" * 60)
print("CIRCUIT 2 - Parameter Estimation")
print("=" * 60)

params_2A, logpost_2A, model_2A = estimate_parameters(freq2, Z2_complex, 'A')
print()
params_2B, logpost_2B, model_2B = estimate_parameters(freq2, Z2_complex, 'B')

In [ ]:
# Visualize fits
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Circuit 1 - Model A
Z1_fit_A = impedance_model_A(freq1, *params_1A)
axes[0, 0].plot(Z1_complex.real, -Z1_complex.imag, 'o', label='Measured', markersize=6, alpha=0.6)
axes[0, 0].plot(Z1_fit_A.real, -Z1_fit_A.imag, 's-', label='Model A Fit', markersize=4)
axes[0, 0].set_xlabel('Real(Z) [Ω]')
axes[0, 0].set_ylabel('-Imag(Z) [Ω]')
axes[0, 0].set_title('Circuit 1 - Model A Fit')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Circuit 1 - Model B
Z1_fit_B = impedance_model_B(freq1, *params_1B)
axes[0, 1].plot(Z1_complex.real, -Z1_complex.imag, 'o', label='Measured', markersize=6, alpha=0.6)
axes[0, 1].plot(Z1_fit_B.real, -Z1_fit_B.imag, '^-', label='Model B Fit', markersize=4)
axes[0, 1].set_xlabel('Real(Z) [Ω]')
axes[0, 1].set_ylabel('-Imag(Z) [Ω]')
axes[0, 1].set_title('Circuit 1 - Model B Fit')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Circuit 2 - Model A
Z2_fit_A = impedance_model_A(freq2, *params_2A)
axes[1, 0].plot(Z2_complex.real, -Z2_complex.imag, 'o', label='Measured', markersize=6, alpha=0.6)
axes[1, 0].plot(Z2_fit_A.real, -Z2_fit_A.imag, 's-', label='Model A Fit', markersize=4)
axes[1, 0].set_xlabel('Real(Z) [Ω]')
axes[1, 0].set_ylabel('-Imag(Z) [Ω]')
axes[1, 0].set_title('Circuit 2 - Model A Fit')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Circuit 2 - Model B
Z2_fit_B = impedance_model_B(freq2, *params_2B)
axes[1, 1].plot(Z2_complex.real, -Z2_complex.imag, 'o', label='Measured', markersize=6, alpha=0.6)
axes[1, 1].plot(Z2_fit_B.real, -Z2_fit_B.imag, '^-', label='Model B Fit', markersize=4)
axes[1, 1].set_xlabel('Real(Z) [Ω]')
axes[1, 1].set_ylabel('-Imag(Z) [Ω]')
axes[1, 1].set_title('Circuit 2 - Model B Fit')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate residuals and goodness of fit
def calculate_residuals(Z_measured, Z_fit):
    residuals = Z_measured - Z_fit
    rmse = np.sqrt(np.mean(np.abs(residuals)**2))
    return residuals, rmse

res_1A, rmse_1A = calculate_residuals(Z1_complex, Z1_fit_A)
res_1B, rmse_1B = calculate_residuals(Z1_complex, Z1_fit_B)
res_2A, rmse_2A = calculate_residuals(Z2_complex, Z2_fit_A)
res_2B, rmse_2B = calculate_residuals(Z2_complex, Z2_fit_B)

print("Residual Analysis:")
print("\nCircuit 1:")
print(f"  Model A RMSE: {rmse_1A:.4f} Ω")
print(f"  Model B RMSE: {rmse_1B:.4f} Ω")
print(f"  Improvement with Model B: {(1 - rmse_1B/rmse_1A)*100:.2f}%")

print("\nCircuit 2:")
print(f"  Model A RMSE: {rmse_2A:.4f} Ω")
print(f"  Model B RMSE: {rmse_2B:.4f} Ω")
print(f"  Improvement with Model B: {(1 - rmse_2B/rmse_2A)*100:.2f}%")

## 6. Model Comparison <a name="model-comparison"></a>

### Bayesian Model Selection

We use the **Bayesian Information Criterion (BIC)** and **Akaike Information Criterion (AIC)** for model comparison:

$$\text{BIC} = -2\ln(\hat{L}) + k\ln(n)$$

$$\text{AIC} = -2\ln(\hat{L}) + 2k$$

where:
- $\hat{L}$ is the maximum likelihood
- $k$ is the number of parameters
- $n$ is the number of data points

Lower BIC/AIC indicates a better model.

We also compute the **Bayes Factor** for direct model comparison:

$$BF_{AB} = \frac{P(D|M_A)}{P(D|M_B)}$$

Using the Bayesian Information Criterion approximation:

$$\ln BF_{AB} \approx \frac{1}{2}(\text{BIC}_B - \text{BIC}_A)$$

In [ ]:
def calculate_model_selection_criteria(log_likelihood, n_params, n_data):
    """
    Calculate AIC and BIC for model selection
    """
    # AIC = -2*log(L) + 2*k
    AIC = -2 * log_likelihood + 2 * n_params
    
    # BIC = -2*log(L) + k*log(n)
    BIC = -2 * log_likelihood + n_params * np.log(n_data)
    
    return AIC, BIC

# Number of parameters
n_params_A = 2  # R, C
n_params_B = 4  # R1, C1, R2, C2
n_data = len(freq1)

# Calculate log-likelihoods at MAP estimates
logL_1A = model_1A.log_likelihood(params_1A)
logL_1B = model_1B.log_likelihood(params_1B)
logL_2A = model_2A.log_likelihood(params_2A)
logL_2B = model_2B.log_likelihood(params_2B)

# Calculate AIC and BIC
AIC_1A, BIC_1A = calculate_model_selection_criteria(logL_1A, n_params_A, n_data)
AIC_1B, BIC_1B = calculate_model_selection_criteria(logL_1B, n_params_B, n_data)
AIC_2A, BIC_2A = calculate_model_selection_criteria(logL_2A, n_params_A, n_data)
AIC_2B, BIC_2B = calculate_model_selection_criteria(logL_2B, n_params_B, n_data)

# Bayes Factor (using BIC approximation)
log_BF_1 = 0.5 * (BIC_1A - BIC_1B)  # log(P(D|A)/P(D|B))
log_BF_2 = 0.5 * (BIC_2A - BIC_2B)

print("=" * 70)
print("MODEL COMPARISON RESULTS")
print("=" * 70)

print("\nCircuit 1:")
print("-" * 70)
print(f"{'Metric':<30} {'Model A':<20} {'Model B':<20}")
print("-" * 70)
print(f"{'Log-Likelihood':<30} {logL_1A:<20.2f} {logL_1B:<20.2f}")
print(f"{'AIC':<30} {AIC_1A:<20.2f} {AIC_1B:<20.2f}")
print(f"{'BIC':<30} {BIC_1A:<20.2f} {BIC_1B:<20.2f}")
print(f"{'RMSE (Ω)':<30} {rmse_1A:<20.4f} {rmse_1B:<20.4f}")
print("-" * 70)
print(f"\nLog Bayes Factor (A vs B): {log_BF_1:.2f}")
if log_BF_1 > 2.5:
    print("  → Strong evidence for Model A")
elif log_BF_1 > 1:
    print("  → Moderate evidence for Model A")
elif log_BF_1 < -2.5:
    print("  → Strong evidence for Model B")
elif log_BF_1 < -1:
    print("  → Moderate evidence for Model B")
else:
    print("  → Inconclusive")

print("\n" + "=" * 70)
print("\nCircuit 2:")
print("-" * 70)
print(f"{'Metric':<30} {'Model A':<20} {'Model B':<20}")
print("-" * 70)
print(f"{'Log-Likelihood':<30} {logL_2A:<20.2f} {logL_2B:<20.2f}")
print(f"{'AIC':<30} {AIC_2A:<20.2f} {AIC_2B:<20.2f}")
print(f"{'BIC':<30} {BIC_2A:<20.2f} {BIC_2B:<20.2f}")
print(f"{'RMSE (Ω)':<30} {rmse_2A:<20.4f} {rmse_2B:<20.4f}")
print("-" * 70)
print(f"\nLog Bayes Factor (A vs B): {log_BF_2:.2f}")
if log_BF_2 > 2.5:
    print("  → Strong evidence for Model A")
elif log_BF_2 > 1:
    print("  → Moderate evidence for Model A")
elif log_BF_2 < -2.5:
    print("  → Strong evidence for Model B")
elif log_BF_2 < -1:
    print("  → Moderate evidence for Model B")
else:
    print("  → Inconclusive")

print("\n" + "=" * 70)

In [ ]:
# Visualization of model comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Circuit 1 - BIC comparison
models = ['Model A', 'Model B']
bic_1 = [BIC_1A, BIC_1B]
aic_1 = [AIC_1A, AIC_1B]

x = np.arange(len(models))
width = 0.35

axes[0, 0].bar(x - width/2, bic_1, width, label='BIC', alpha=0.8)
axes[0, 0].bar(x + width/2, aic_1, width, label='AIC', alpha=0.8)
axes[0, 0].set_ylabel('Information Criterion')
axes[0, 0].set_title('Circuit 1 - Model Comparison (lower is better)')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(models)
axes[0, 0].legend()
axes[0, 0].grid(axis='y', alpha=0.3)

# Circuit 2 - BIC comparison
bic_2 = [BIC_2A, BIC_2B]
aic_2 = [AIC_2A, AIC_2B]

axes[0, 1].bar(x - width/2, bic_2, width, label='BIC', alpha=0.8)
axes[0, 1].bar(x + width/2, aic_2, width, label='AIC', alpha=0.8)
axes[0, 1].set_ylabel('Information Criterion')
axes[0, 1].set_title('Circuit 2 - Model Comparison (lower is better)')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(models)
axes[0, 1].legend()
axes[0, 1].grid(axis='y', alpha=0.3)

# Residual plots
axes[1, 0].plot(freq1[1:], np.abs(res_1A[1:]), 'o-', label='Model A', alpha=0.6)
axes[1, 0].plot(freq1[1:], np.abs(res_1B[1:]), 's-', label='Model B', alpha=0.6)
axes[1, 0].set_xlabel('Frequency [Hz]')
axes[1, 0].set_ylabel('|Residual| [Ω]')
axes[1, 0].set_title('Circuit 1 - Residuals')
axes[1, 0].set_xscale('log')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(freq2[1:], np.abs(res_2A[1:]), 'o-', label='Model A', alpha=0.6)
axes[1, 1].plot(freq2[1:], np.abs(res_2B[1:]), 's-', label='Model B', alpha=0.6)
axes[1, 1].set_xlabel('Frequency [Hz]')
axes[1, 1].set_ylabel('|Residual| [Ω]')
axes[1, 1].set_title('Circuit 2 - Residuals')
axes[1, 1].set_xscale('log')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Sensor Uncertainties and Error Propagation <a name="uncertainties"></a>

### Measurement Uncertainties

We estimate measurement uncertainties from:
1. Residual analysis (model fit errors)
2. Frequency-dependent noise characteristics

### Error Propagation

For parameter uncertainties, we use the Hessian matrix at the MAP estimate:

$$\Sigma_{\theta} \approx \left[-\nabla^2 \log P(\theta|D)\right]^{-1}$$

This gives us the covariance matrix of parameter estimates.

In [ ]:
from scipy.optimize import approx_fprime

def estimate_parameter_uncertainties(model, params_opt, epsilon=1e-8):
    """
    Estimate parameter uncertainties using Hessian approximation
    """
    n_params = len(params_opt)
    
    # Approximate Hessian using finite differences
    def neg_log_post(p):
        return -model.log_posterior(p)
    
    # Calculate Hessian
    hessian = np.zeros((n_params, n_params))
    for i in range(n_params):
        # Numerical gradient
        def grad_i(p):
            return approx_fprime(p, neg_log_post, epsilon)[i]
        
        hessian[i, :] = approx_fprime(params_opt, grad_i, epsilon)
    
    # Covariance is inverse of Hessian
    try:
        covariance = np.linalg.inv(hessian)
        uncertainties = np.sqrt(np.diag(covariance))
    except:
        print("  Warning: Hessian is singular, using approximate uncertainties")
        uncertainties = np.ones(n_params) * 0.1 * np.abs(params_opt)
        covariance = np.diag(uncertainties**2)
    
    return uncertainties, covariance

print("Estimating parameter uncertainties...\n")

# Circuit 1
print("Circuit 1 - Model A:")
unc_1A, cov_1A = estimate_parameter_uncertainties(model_1A, params_1A)
print(f"  R = {params_1A[0]:.2f} ± {unc_1A[0]:.2f} Ω")
print(f"  C = {params_1A[1]*1e9:.2f} ± {unc_1A[1]*1e9:.2f} nF")

print("\nCircuit 1 - Model B:")
unc_1B, cov_1B = estimate_parameter_uncertainties(model_1B, params_1B)
print(f"  R1 = {params_1B[0]:.2f} ± {unc_1B[0]:.2f} Ω")
print(f"  C1 = {params_1B[1]*1e9:.2f} ± {unc_1B[1]*1e9:.2f} nF")
print(f"  R2 = {params_1B[2]:.2f} ± {unc_1B[2]:.2f} Ω")
print(f"  C2 = {params_1B[3]*1e9:.2f} ± {unc_1B[3]*1e9:.2f} nF")

# Circuit 2
print("\nCircuit 2 - Model A:")
unc_2A, cov_2A = estimate_parameter_uncertainties(model_2A, params_2A)
print(f"  R = {params_2A[0]:.2f} ± {unc_2A[0]:.2f} Ω")
print(f"  C = {params_2A[1]*1e9:.2f} ± {unc_2A[1]*1e9:.2f} nF")

print("\nCircuit 2 - Model B:")
unc_2B, cov_2B = estimate_parameter_uncertainties(model_2B, params_2B)
print(f"  R1 = {params_2B[0]:.2f} ± {unc_2B[0]:.2f} Ω")
print(f"  C1 = {params_2B[1]*1e9:.2f} ± {unc_2B[1]*1e9:.2f} nF")
print(f"  R2 = {params_2B[2]:.2f} ± {unc_2B[2]:.2f} Ω")
print(f"  C2 = {params_2B[3]*1e9:.2f} ± {unc_2B[3]*1e9:.2f} nF")

In [ ]:
# Estimate noise characteristics
def analyze_noise_characteristics(frequencies, residuals):
    """
    Analyze frequency-dependent noise
    """
    residuals_mag = np.abs(residuals)
    
    # Overall statistics
    mean_residual = np.mean(residuals_mag)
    std_residual = np.std(residuals_mag)
    
    return mean_residual, std_residual, residuals_mag

print("\nNoise Analysis:")
print("=" * 60)

mean_1A, std_1A, _ = analyze_noise_characteristics(freq1, res_1A)
mean_1B, std_1B, _ = analyze_noise_characteristics(freq1, res_1B)
mean_2A, std_2A, _ = analyze_noise_characteristics(freq2, res_2A)
mean_2B, std_2B, _ = analyze_noise_characteristics(freq2, res_2B)

print(f"\nCircuit 1 - Model A: μ = {mean_1A:.4f} Ω, σ = {std_1A:.4f} Ω")
print(f"Circuit 1 - Model B: μ = {mean_1B:.4f} Ω, σ = {std_1B:.4f} Ω")
print(f"\nCircuit 2 - Model A: μ = {mean_2A:.4f} Ω, σ = {std_2A:.4f} Ω")
print(f"Circuit 2 - Model B: μ = {mean_2B:.4f} Ω, σ = {std_2B:.4f} Ω")

## 8. Data Fusion and Architecture <a name="data-fusion"></a>

### Sensor Fusion Architecture

In this problem, we have two independent measurements (Circuit1 and Circuit2). We can apply sensor fusion to:
1. Improve parameter estimation precision
2. Reduce uncertainty
3. Make more robust model selection decisions

### Fusion Architectures Applied:

1. **Data-level fusion (Low-level):** Combining raw impedance measurements
2. **Feature-level fusion (Mid-level):** Combining parameter estimates
3. **Decision-level fusion (High-level):** Combining model selection decisions

### Bayesian Fusion

For independent measurements:

$$P(\theta | D_1, D_2) \propto P(D_1 | \theta) P(D_2 | \theta) P(\theta)$$

The fused posterior combines information from both circuits.

In [ ]:
class FusedBayesianModel:
    """
    Bayesian model fusion for multiple circuits
    """
    
    def __init__(self, models_list):
        """
        Initialize with list of BayesianCircuitModel instances
        """
        self.models = models_list
        self.model_type = models_list[0].model_type
        
    def log_likelihood_fused(self, params):
        """
        Combined log-likelihood from all circuits
        """
        log_L_total = 0
        for model in self.models:
            log_L_total += model.log_likelihood(params)
        return log_L_total
    
    def log_prior(self, params):
        """
        Use prior from first model (same for all)
        """
        return self.models[0].log_prior(params)
    
    def log_posterior_fused(self, params):
        """
        Fused posterior
        """
        return self.log_prior(params) + self.log_likelihood_fused(params)
    
    def negative_log_posterior_fused(self, params):
        return -self.log_posterior_fused(params)

# Create fused models
print("Performing Bayesian Sensor Fusion...\n")

fused_model_A = FusedBayesianModel([model_1A, model_2A])
fused_model_B = FusedBayesianModel([model_1B, model_2B])

# Optimize fused models
if fused_model_A.model_type == 'A':
    bounds_A = [(100, 10000), (10e-9, 10e-6)]
else:
    bounds_A = [(100, 10000), (10e-9, 10e-6), (100, 10000), (10e-9, 10e-6)]

if fused_model_B.model_type == 'B':
    bounds_B = [(100, 10000), (10e-9, 10e-6), (100, 10000), (10e-9, 10e-6)]
else:
    bounds_B = [(100, 10000), (10e-9, 10e-6)]

result_fused_A = differential_evolution(
    fused_model_A.negative_log_posterior_fused,
    bounds=bounds_A,
    maxiter=1000,
    seed=42,
    polish=True
)

result_fused_B = differential_evolution(
    fused_model_B.negative_log_posterior_fused,
    bounds=bounds_B,
    maxiter=1000,
    seed=42,
    polish=True
)

params_fused_A = result_fused_A.x
params_fused_B = result_fused_B.x
logpost_fused_A = -result_fused_A.fun
logpost_fused_B = -result_fused_B.fun

print("Fused Model A (combining Circuit 1 & 2):")
print(f"  R = {params_fused_A[0]:.2f} Ω")
print(f"  C = {params_fused_A[1]*1e9:.2f} nF")
print(f"  Log posterior: {logpost_fused_A:.2f}")

print("\nFused Model B (combining Circuit 1 & 2):")
print(f"  R1 = {params_fused_B[0]:.2f} Ω")
print(f"  C1 = {params_fused_B[1]*1e9:.2f} nF")
print(f"  R2 = {params_fused_B[2]:.2f} Ω")
print(f"  C2 = {params_fused_B[3]*1e9:.2f} nF")
print(f"  Log posterior: {logpost_fused_B:.2f}")

# Model comparison for fused data
n_data_fused = len(freq1) + len(freq2)
logL_fused_A = fused_model_A.log_likelihood_fused(params_fused_A)
logL_fused_B = fused_model_B.log_likelihood_fused(params_fused_B)

AIC_fused_A, BIC_fused_A = calculate_model_selection_criteria(logL_fused_A, 2, n_data_fused)
AIC_fused_B, BIC_fused_B = calculate_model_selection_criteria(logL_fused_B, 4, n_data_fused)

log_BF_fused = 0.5 * (BIC_fused_A - BIC_fused_B)

print("\n" + "=" * 60)
print("FUSED MODEL COMPARISON")
print("=" * 60)
print(f"\n{'Metric':<30} {'Model A':<20} {'Model B':<20}")
print("-" * 70)
print(f"{'Log-Likelihood':<30} {logL_fused_A:<20.2f} {logL_fused_B:<20.2f}")
print(f"{'AIC':<30} {AIC_fused_A:<20.2f} {AIC_fused_B:<20.2f}")
print(f"{'BIC':<30} {BIC_fused_A:<20.2f} {BIC_fused_B:<20.2f}")
print("-" * 70)
print(f"\nLog Bayes Factor (A vs B): {log_BF_fused:.2f}")
if log_BF_fused > 2.5:
    print("  → Strong evidence for Model A")
    print("  → Circuit 1 and 2 are CIRCUIT A (single RC parallel)")
elif log_BF_fused > 1:
    print("  → Moderate evidence for Model A")
    print("  → Circuit 1 and 2 are likely CIRCUIT A")
elif log_BF_fused < -2.5:
    print("  → Strong evidence for Model B")
    print("  → Circuit 1 and 2 are CIRCUIT B (dual RC parallel)")
elif log_BF_fused < -1:
    print("  → Moderate evidence for Model B")
    print("  → Circuit 1 and 2 are likely CIRCUIT B")
else:
    print("  → Inconclusive")

### Sensor Fusion Architecture Classification

Our implementation uses multiple fusion architectures:

1. **Centralized Architecture:** All data is sent to a central processor (this notebook) for fusion
   - Optimal for our case with only 2 circuits
   - Provides best performance when communication is not limited

2. **Data-level Fusion:** We combine raw impedance measurements from both circuits
   - Preserves maximum information
   - Used in fused likelihood calculation

3. **Decision-level Fusion:** We can also combine model selection decisions
   - More robust to outliers
   - Useful when circuits may have different characteristics

## 9. Critical Analysis and Sensitivity <a name="sensitivity"></a>

### Sensitivity to Prior Assumptions

We analyze how sensitive our results are to the choice of priors.

In [ ]:
# Sensitivity analysis: vary prior bounds
def sensitivity_to_priors(frequencies, Z_measured, model_type, 
                         R_bounds_list, C_bounds_list):
    """
    Test sensitivity to prior bounds
    """
    results = []
    
    for R_bounds, C_bounds in zip(R_bounds_list, C_bounds_list):
        # Create model with modified priors
        model = BayesianCircuitModel(frequencies, Z_measured, model_type)
        model.R_min, model.R_max = R_bounds
        model.C_min, model.C_max = C_bounds
        
        # Optimize
        if model_type == 'A':
            bounds = [R_bounds, C_bounds]
        else:
            bounds = [R_bounds, C_bounds, R_bounds, C_bounds]
        
        result = differential_evolution(
            lambda params: model.negative_log_posterior(params),
            bounds=bounds,
            maxiter=500,
            seed=42,
            polish=True
        )
        
        results.append({
            'R_bounds': R_bounds,
            'C_bounds': C_bounds,
            'params': result.x,
            'log_post': -result.fun
        })
    
    return results

# Define different prior scenarios
R_scenarios = [
    (100, 10000),      # Original
    (50, 20000),       # Wider
    (500, 5000),       # Narrower
]

C_scenarios = [
    (10e-9, 10e-6),    # Original
    (1e-9, 100e-6),    # Wider
    (50e-9, 5e-6),     # Narrower
]

print("Sensitivity Analysis - Prior Bounds")
print("=" * 70)

print("\nCircuit 1 - Model A:")
sens_1A = sensitivity_to_priors(freq1, Z1_complex, 'A', R_scenarios, C_scenarios)
for i, res in enumerate(sens_1A):
    print(f"\nScenario {i+1}:")
    print(f"  R bounds: {res['R_bounds']}")
    print(f"  C bounds: {tuple(c*1e9 for c in res['C_bounds'])} nF")
    print(f"  R = {res['params'][0]:.2f} Ω")
    print(f"  C = {res['params'][1]*1e9:.2f} nF")
    print(f"  Log posterior: {res['log_post']:.2f}")

# Check parameter consistency
R_estimates = [res['params'][0] for res in sens_1A]
C_estimates = [res['params'][1] for res in sens_1A]

print(f"\nParameter Variability:")
print(f"  R: {np.std(R_estimates)/np.mean(R_estimates)*100:.2f}% coefficient of variation")
print(f"  C: {np.std(C_estimates)/np.mean(C_estimates)*100:.2f}% coefficient of variation")

### Critical Discussion

**Assumptions:**
1. **Gaussian noise:** We assume measurement errors are Gaussian distributed
2. **Independent measurements:** Frequency points are assumed independent
3. **Log-uniform priors:** Appropriate for scale parameters spanning orders of magnitude
4. **Model completeness:** We only consider two models (A and B)

**Limitations:**
1. Parasitic elements (inductance, stray capacitance) not modeled
2. Frequency-dependent component values not considered
3. Temperature effects not modeled
4. Limited to two candidate models

**Strengths:**
1. Rigorous Bayesian framework for model comparison
2. Uncertainty quantification for parameters
3. Multiple information criteria used (AIC, BIC, Bayes Factor)
4. Sensor fusion improves robustness

## 10. Conclusions <a name="conclusions"></a>

### Summary of Findings

In [ ]:
print("=" * 70)
print("FINAL SUMMARY AND CONCLUSIONS")
print("=" * 70)

print("\n1. MODEL SELECTION RESULTS:")
print("-" * 70)

# Determine best model for each circuit
if BIC_1A < BIC_1B:
    print("\n  Circuit 1: Model A (Single RC Parallel)")
    print(f"    Evidence: ΔBIC = {BIC_1B - BIC_1A:.2f}")
else:
    print("\n  Circuit 1: Model B (Dual RC Parallel)")
    print(f"    Evidence: ΔBIC = {BIC_1A - BIC_1B:.2f}")

if BIC_2A < BIC_2B:
    print("\n  Circuit 2: Model A (Single RC Parallel)")
    print(f"    Evidence: ΔBIC = {BIC_2B - BIC_2A:.2f}")
else:
    print("\n  Circuit 2: Model B (Dual RC Parallel)")
    print(f"    Evidence: ΔBIC = {BIC_2A - BIC_2B:.2f}")

print("\n  Fused Analysis (Combined Data):")
if BIC_fused_A < BIC_fused_B:
    print("    Model A (Single RC Parallel)")
    print(f"    Evidence: ΔBIC = {BIC_fused_B - BIC_fused_A:.2f}")
else:
    print("    Model B (Dual RC Parallel)")
    print(f"    Evidence: ΔBIC = {BIC_fused_A - BIC_fused_B:.2f}")

print("\n2. ESTIMATED PARAMETERS:")
print("-" * 70)
print("\n  Best-fit parameters (fused analysis):")
if BIC_fused_A < BIC_fused_B:
    print(f"\n    Model A:")
    print(f"      R = {params_fused_A[0]:.2f} Ω")
    print(f"      C = {params_fused_A[1]*1e9:.2f} nF")
else:
    print(f"\n    Model B:")
    print(f"      R1 = {params_fused_B[0]:.2f} Ω")
    print(f"      C1 = {params_fused_B[1]*1e9:.2f} nF")
    print(f"      R2 = {params_fused_B[2]:.2f} Ω")
    print(f"      C2 = {params_fused_B[3]*1e9:.2f} nF")

print("\n3. METHODOLOGY:")
print("-" * 70)
print("  ✓ Bayesian inference with log-uniform priors")
print("  ✓ Maximum a posteriori (MAP) estimation")
print("  ✓ Model comparison using BIC, AIC, and Bayes Factors")
print("  ✓ Uncertainty quantification via Hessian approximation")
print("  ✓ Sensor fusion for improved precision")
print("  ✓ Sensitivity analysis for robustness")

print("\n4. SENSOR FUSION ARCHITECTURES APPLIED:")
print("-" * 70)
print("  • Centralized architecture (all data processed centrally)")
print("  • Data-level fusion (combining raw measurements)")
print("  • Feature-level fusion (combining parameter estimates)")
print("  • Decision-level fusion (combining model selection results)")

print("\n" + "=" * 70)

### Final Recommendations

Based on the Bayesian analysis:

1. **Model Selection:** The model with lower BIC/AIC and higher Bayes Factor is preferred
   - BIC penalizes model complexity more heavily than AIC
   - Bayes Factor provides direct probabilistic interpretation

2. **Parameter Estimates:** Use fused estimates for best precision
   - Combining data from both circuits reduces uncertainty
   - Covariance matrices quantify parameter correlations

3. **Uncertainty Quantification:** Report parameters with confidence intervals
   - Hessian-based uncertainties are approximate
   - MCMC sampling would provide full posterior distributions (computational cost)

4. **Future Improvements:**
   - Consider additional circuit models
   - Use MCMC (e.g., emcee, PyMC) for full posterior sampling
   - Model frequency-dependent noise
   - Include parasitic elements if data shows deviations